In [ ]:
!pip install -q gradio

In [ ]:
import torch
from PIL import Image
import gradio as gr
from diffusers import StableDiffusionImg2ImgPipeline


device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
).to(device)


pipe.enable_attention_slicing()



In [ ]:
import gradio as gr
from PIL import Image

 
PROMPT_OPTIONS = {
    "Corroded | Pipeline in the ocean":
        "An old rusted pipeline submerged in the sea, encrusted with marine growth and decay, surrounded by dark water and shafts of light from the surface, cinematic, moody atmosphere",

    "Corroded | Pipeline fading":
        "A decaying underwater pipeline stretching into the abyss, corroded metal flaking away, illuminated by faint sunbeams breaking through deep blue water, mysterious and haunting",

    "Corroded | Coral-covered industrial pipe":
        "An old industrial pipe overgrown with coral and sponges, rust bleeding into the water, quiet deep ocean ambiance with faint beams of surface light",
}

 
STRENGTH = 0.75
GUIDANCE_SCALE = 7.5

 
def generate_image(init_image, prompt_choice):
    init_image = init_image.convert("RGB").resize((768, 512))
    prompt = PROMPT_OPTIONS[prompt_choice]

    result = pipe(
        prompt=prompt,
        image=init_image,
        strength=STRENGTH,
        guidance_scale=GUIDANCE_SCALE
    ).images[0]

    return result


with gr.Blocks(css="""
    .gr-block { padding: 1rem !important; }
    .gr-button { width: 100%; font-weight: bold; }
    .gr-image { border-radius: 12px; }
""") as demo:
    
    gr.Markdown("""
        <h1 style='text-align: center; color: #333;'> Corroded Pipeline Image Generator</h1>
        <p style='text-align: center; max-width: 700px; margin: auto; color: #555;'>
            Upload an initial image and choose a corrosion style prompt to generate an image.
        </p>
    """)

    with gr.Row():
        with gr.Column(scale=1):
            init_image = gr.Image(
                label="Upload Your Image",
                type="pil",
                height=300,
                width=400
            )
            prompt_choice = gr.Dropdown(
                choices=list(PROMPT_OPTIONS.keys()),
                label="Select Scene Type",
                value="Corroded | Pipeline in the ocean"
            )
            generate_btn = gr.Button("Generate Image")
        
        with gr.Column(scale=1):
            output_image = gr.Image(label="Generated Image", height=300)

    generate_btn.click(
        fn=generate_image,
        inputs=[init_image, prompt_choice],
        outputs=output_image
    )

demo.launch(share=True)
